In [ ]:
%%capture
%pip install -q -U google-genai

In [2]:
from google import genai
from google.genai import types
import pandas as pd
import numpy as np
import time
from google.genai.types import CreateBatchJobConfig, JobState, HttpOptions

client = genai.Client(http_options=HttpOptions(api_version="v1"),
                      vertexai=True,
                      project="gen-lang-client-0712587895",
                      location="us-central1")

In [ ]:
# models = client.models.list()
# for model in models:
#     model = model.__dict__
#     print(model['name'])

In [ ]:
# import json
# df = pd.read_csv("./data/test.csv").reset_index(drop=True)
# batches = np.array_split(df["abstract"], 5)

# instruction = "Write a consise, formal and structured title for this scientific research work, return ONLY the title:\n"

# for i, batch in enumerate(batches):
#     with open(f"./batches/gemini_batch_input_{i+1}.jsonl", "w") as f:
#         for line in batch:
#             entry = {
#                 "request":
#                     {"contents": [
#                             {"role": "user",
#                             "parts": [
#                                 {"text": instruction + line}
#                                 ]
#                             }
#                         ]
#                     }
#                 }
#             f.write(json.dumps(entry) + "\n")


In [ ]:
!gcloud projects add-iam-policy-binding gen-lang-client-0712587895 \
  --member="serviceAccount:gemini-api@gen-lang-client-0712587895.iam.gserviceaccount.com" \
  --role="roles/aiplatform.user"

# Replace the `GOOGLE_CLOUD_PROJECT` and `GOOGLE_CLOUD_LOCATION` values
# with appropriate values for your project.
!export GOOGLE_CLOUD_PROJECT=gen-lang-client-0712587895
!export GOOGLE_CLOUD_LOCATION=us-central1
!export GOOGLE_GENAI_USE_VERTEXAI=True

In [ ]:
output_uri = "gs://gemini_batch_input/output"
batch_jobs = []


for i in range(1,6):
    # See the documentation: https://googleapis.github.io/python-genai/genai.html#genai.batches.Batches.create
    job = client.batches.create(
        model="gemini-2.0-flash-001",
        # Source link: https://storage.cloud.google.com/cloud-samples-data/batch/prompt_for_batch_gemini_predict.jsonl
        src=f"gs://gemini_batch_input/gemini_batch_input_{i}.jsonl",
        config=CreateBatchJobConfig(dest=output_uri),
    )
    batch_jobs.append(job.name)

In [ ]:
with open("./batches/gemini_batch_jobs.txt", "w") as f:
    for bj in batch_jobs:
        print(bj)
        f.write(bj + "\n")

In [3]:
for job in client.batches.list(config=types.ListBatchJobsConfig(page_size=10)):
    print(job)
# for job_id in batch_jobs:
#     job = client.batches.get(name=job_id)
#     print(job)

name='projects/32693730648/locations/us-central1/batchPredictionJobs/2667732022952198144' display_name='genai_batch_job_20250417232732_a08aa' state=<JobState.JOB_STATE_SUCCEEDED: 'JOB_STATE_SUCCEEDED'> error=None create_time=datetime.datetime(2025, 4, 17, 16, 27, 33, 673067, tzinfo=TzInfo(UTC)) start_time=datetime.datetime(2025, 4, 17, 16, 52, 59, 246125, tzinfo=TzInfo(UTC)) end_time=datetime.datetime(2025, 4, 17, 16, 57, 27, 279260, tzinfo=TzInfo(UTC)) update_time=datetime.datetime(2025, 4, 17, 16, 57, 27, 279260, tzinfo=TzInfo(UTC)) model='publishers/google/models/gemini-2.0-flash-001' src=BatchJobSource(format='jsonl', gcs_uri=['gs://gemini_batch_input/gemini_batch_input_5.jsonl'], bigquery_uri=None) dest=BatchJobDestination(format='jsonl', gcs_uri='gs://gemini_batch_input/output', bigquery_uri=None)
name='projects/32693730648/locations/us-central1/batchPredictionJobs/181745028643684352' display_name='genai_batch_job_20250417232732_c3721' state=<JobState.JOB_STATE_SUCCEEDED: 'JOB_ST

In [ ]:
# See the documentation: https://googleapis.github.io/python-genai/genai.html#genai.types.BatchJob
completed_states = {
    JobState.JOB_STATE_SUCCEEDED,
    JobState.JOB_STATE_FAILED,
    JobState.JOB_STATE_CANCELLED,
    JobState.JOB_STATE_PAUSED,
}

for job_id in batch_jobs:
    job = client.batches.get(name=job_id)
    while job.state not in completed_states:
        time.sleep(30)
        job = client.batches.get(name=job.name)
        print(f"Job state: {job.state}")

In [21]:
import os
import json
import pandas as pd
df = pd.DataFrame()
files = [f for f in os.listdir("./output/") if f.startswith("output_prediction")]



def file_2_df(file):
    requests = []
    responses = []

    with open("./output/" + file, "r") as f:
        for line in f:
            line = json.loads(line.strip())
            request = line['request']['contents'][0]['parts'][0]['text']
            response = line['response']['candidates'][0]['content']['parts'][0]['text']
            requests.append(request)
            responses.append(response)
        
    df = pd.DataFrame({'request': requests, 'response': responses})
    return df

for file in files:
    df = pd.concat([df, file_2_df(file)], ignore_index=True)

pd.set_option('display.max_colwidth',True)
df.shape

(5701, 2)

In [26]:
instruction = "Write a consise, formal and structured title for this scientific research work, return ONLY the title:\n"
test_df = pd.read_csv("./data/test.csv")['abstract']

# Remove the instruction prefix from df's request column
df['abstract'] = df['request'].str.replace(instruction, '', regex=False)

# Merge df with test_df based on the abstract content
test_df = test_df.to_frame(name='abstract')
merged_df = test_df.merge(df, on='abstract', how='left')

# Drop the abstract column from the merged dataframe
merged_df = merged_df.drop(columns=['abstract'])
merged_df = merged_df.rename(columns={'response': 'title'})

merged_df['title'] = merged_df['title'].str.strip('\n"')
merged_df['title'] = merged_df['title'].str.lstrip('"')

merged_df['title'].to_csv("./output/gemini-2.0-flash-001-output.csv", index=False)